In [90]:
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.metrics import accuracy_score, f1_score

In [16]:
root_path = 'C:\\Users\\User\\Desktop\\Data_Science\\Projects\\HW_27_Image_Generators\\data_set_cat_dogs'
cat_folder = 'C:\\Users\\User\\Desktop\\Data_Science\\Projects\\HW_27_Image_Generators\\data_set_cat_dogs\\cat'
dog_folder = 'C:\\Users\\User\\Desktop\\Data_Science\\Projects\\HW_27_Image_Generators\\data_set_cat_dogs\\dog'

X, y_ = [], []

for img_name in os.listdir(cat_folder):
   img_path = os.path.join(cat_folder, img_name)
   img = Image.open(img_path).resize((256, 256)) 
   X.append(np.array(img)) 
   y_.append(0)  
    
for img_name in os.listdir(dog_folder):
   img_path = os.path.join(dog_folder, img_name)
   img = Image.open(img_path).resize((256, 256))  
   X.append(np.array(img))  
   y_.append(1) 
   
X = np.array(X)
y_ = np.array(y_)

y_ = to_categorical(y_)

X, y_ = shuffle(X, y_, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y_, test_size=0.3)

In [96]:
class Augmentator(BaseEstimator, TransformerMixin):
    def __init__(self, aug=False):
        
        self.aug = aug
        self.final_generator = None
        self.generator = None
        self.y = None

    def fit(self, X, y=None):
        if self.aug:
            self.generator = ImageDataGenerator(rescale=1./255,
                                           rotation_range=40,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           zoom_range=0.2,
                                           shear_range=0.2,
                                           horizontal_flip=True,
                                           fill_mode='nearest')

           
        else:
            self.generator = ImageDataGenerator(rescale=1./255)
        
        
        return self

    def transform(self, X, y=None):
        self.final_generator = self.generator.flow(X, y, batch_size=64, shuffle=True, seed=123)

        return self.final_generator


class Model(BaseEstimator, TransformerMixin):
    def __init__(self,  model_name='ResNet'):
        self.model_name = model_name
        self.model = None

    def fit(self, X, y=None):

        if self.model_name == 'ResNet':
            base_model = ResNet152V2(include_top=False, input_shape=(256, 256, 3))
        else:
            base_model = VGG16(include_top=False, input_shape=(256, 256, 3))

        base_model.trainable = False
        
        model = Sequential([
            base_model,
            Flatten(),
            Dense(2, activation="softmax")
        ])
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        history = model.fit(X, epochs=1)
        self.model = model
        
        return self

    def transform(self, X):

        return X
        
    def predict(self, X):
        predicted = []
        num_batches = len(X)
        for i in range(num_batches): 
            x_batch= X[i]
            predictions = self.model.predict(x_batch)
            predicted.extend(np.argmax(predictions, axis=1))
        return np.array(predicted)

   
    def score(self, X, y=None): 

        if self.model is None:
            raise ValueError("The model is not trained")  
        predicted = []
        true_labels = []
        num_batches = len(X)
        for i in range(num_batches): 
            x_batch, y_batch = X[i]
            predictions = self.model.predict(x_batch)
            predicted.extend(np.argmax(predictions, axis=1))
            true_labels.extend(np.argmax(y_batch, axis=1))
            
        return accuracy_score(true_labels, predicted), f1_score(true_labels, predicted)

In [98]:
pipe = Pipeline([
    ('augmentator', Augmentator(aug=True)), 
    ('model', Model())
])

pipe.fit((X_train, y_train))

C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 263s 18s/step - accuracy: 0.8092 - loss: 1.2385


Pipeline(steps=[('augmentator', Augmentator(aug=True)), ('model', Model())])

In [100]:
pipe.score((X_test, y_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 29s 11s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 10s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 10s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 22s 11s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 14s/step


(0.9737704918032787, 0.9705882352941176)

In [101]:
pipe.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 22s 11s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 10s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 9s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 10s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 6s/step


array([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,